In [11]:
import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf
import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from math import radians, cos, sin, asin, sqrt
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

from math import radians, cos, sin, asin, sqrt

import copy
import sys

import geopandas as gpd
import numpy as np

import gpflow


code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood
gpflow.config.default_float()


from zinf_gp.metrics import normcdf, fixed_top_X

from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt


In [4]:
run_template = '{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}_{lr}'
#/annual_group_normal_2000_-auto_200_0.01

In [5]:
time = 'annual'
loc = 'tract'
model = 'normal'
start_year = '2010'
cov = '-auto'
num_inducing = '400'
test_years=2
data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/new_big_run'

In [6]:
town_map = pd.read_csv(os.path.join(data_dir, 'town_tract_map.csv'), dtype=str)
group_map = gpd.read_file(os.path.join(data_dir, 'tract_group_map'), dtype=str)

# test y always comes from quarterly tract
y_timesteps_per_year = 1
file_name = f'clean_annual_tract'
data_path = os.path.join(data_dir, file_name)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile',
              'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat', 'lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

#data_gdf = gpd.read_file(data_path)

last_train_year = 2018

test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year + test_years)][y_idx_cols]
starting_y_timestep = int(test_y[timestep_col].min())

sorted_y_timesteps = test_y[timestep_col].unique()
sorted_y_timesteps.sort()

x_timesteps_per_year = {'quarter': 4, 'semi': 2, 'annual': 1}[time]

file_name = f'clean_{time}_{loc}'
data_path = os.path.join(data_dir, file_name)

#data_gdf = gpd.read_file(data_path)

test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year + test_years)][x_idx_cols]

starting_x_timestep = int(test_x[timestep_col].min())

test_timesteps_per_year = max(y_timesteps_per_year, x_timesteps_per_year)
test_timesteps = test_timesteps_per_year * test_years

x_repeats = int(test_timesteps_per_year / x_timesteps_per_year)
y_repeats = int(test_timesteps_per_year / y_timesteps_per_year)

sorted_x_timesteps = test_x[timestep_col].unique()
sorted_x_timesteps.sort()

x_timesteps = [timestep for timestep in sorted_x_timesteps for _ in range(x_repeats)]

y_timesteps = [timestep for timestep in sorted_y_timesteps for _ in range(y_repeats)]

In [7]:
best_elbo = -np.inf
for lr in ['0.01','0.001']:
    for model in ['normal', 'poisson']:
        for start_year in ['2000','2010']:
            for cov in ['-auto','all']:
                for num_inducing in ['200','400']:

                    this_run = run_template.format(time=time, loc=loc,
                                                   model=model, start_year=start_year,
                                                   cov=cov,
                                                   num_inducing=num_inducing, lr=lr)
                    try:
                        with open(os.path.join(log_dir, this_run, 'model.mod'), 'rb') as f:
                            predictor = pickle.load(f)
                        with open(os.path.join(log_dir, this_run, 'stats.csv'), 'rb') as f:
                            stats = pd.read_csv(f)
                            elbo = stats.iloc[-1, :][['elbo']].values[0]
                    except(FileNotFoundError):
                        print(os.path.join(log_dir,this_run))
                        continue

                    if elbo > best_elbo:
                        best_elbo = elbo
                        print(this_run)
                        best_predictor = copy.deepcopy(predictor)

2023-06-15 16:06:45.834797: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-15 16:06:45.834837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2023-06-15 16:06:45.834845: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: p1cmp078.pax.tufts.edu
2023-06-15 16:06:45.835030: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 515.65.1
2023-06-15 16:06:45.835059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.65.1
2023-06-15 16:06:45.835066: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 515.65.1


annual_tract_normal_2000_-auto_200_0.01
annual_tract_normal_2000_-auto_400_0.01
annual_tract_normal_2010_-auto_200_0.01
annual_tract_normal_2010_-auto_400_0.01
annual_tract_poisson_2000_-auto_200_0.01
annual_tract_poisson_2000_-auto_400_0.01
annual_tract_poisson_2010_-auto_200_0.01
annual_tract_poisson_2010_-auto_400_0.01


In [8]:
xtops = []
for year in range(test_years):
    xtop_year = []
    max_timesteps = max(x_timesteps_per_year, y_timesteps_per_year)
    for x_time, y_time in zip(x_timesteps[year * max_timesteps:(year + 1) * max_timesteps],
                              y_timesteps[year * max_timesteps:(year + 1) * max_timesteps]):
        test_x_time = test_x[test_x[timestep_col] == x_time]
        test_y_time = test_y[test_y[timestep_col] == y_time]
        _, _, _, fmean, fvar, gmean, gvar, _, _ = best_predictor.build_predict(test_x_time.loc[:, features_only].values)
        g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
        pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])


        y_index = test_y_time[geography_col]
        sampled_size = int(0.95*len(y_index))
        assert(sampled_size==1539)
        sampled_xtops = []
        for _ in range(100):
            sampled_index = np.random.choice(y_index, sampled_size,replace=False)
            sampled_test = test_y_time.set_index(geography_col).loc[sampled_index, outcome_col]
            sampled_pred = pred_df.loc[sampled_index]
            result = fixed_top_X(sampled_test, sampled_pred, 100)[-1]
            sampled_xtops.append(result)

        xtop_ptiles = np.percentile(sampled_xtops, [2.5, 50, 97.5])
        xtop_year.append(xtop_ptiles)
    annual_avg = np.array(xtop_year).mean(axis=0)
    xtops.append(annual_avg)

In [9]:
max_timesteps

1

In [10]:
xtops

[array([0.44046947, 0.45405563, 0.47257506]),
 array([0.43850162, 0.45537507, 0.48084863])]

In [18]:
xtops = []
maes = []
rmses = []
for year in range(test_years):
    xtop_year = []
    max_timesteps = max(x_timesteps_per_year, y_timesteps_per_year)
    for x_time, y_time in zip(x_timesteps[year * max_timesteps:(year + 1) * max_timesteps],
                              y_timesteps[year * max_timesteps:(year + 1) * max_timesteps]):
        test_x_time = test_x[test_x[timestep_col] == x_time]
        test_y_time = test_y[test_y[timestep_col] == y_time]
        _, _, _, fmean, fvar, gmean, gvar, _, _ = best_predictor.build_predict(test_x_time.loc[:, features_only].values)
        g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
        pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])

        result = fixed_top_X(test_y_time.set_index(geography_col).loc[:, outcome_col], pred_df, 100)[-1]
        
        rmses.append(sqrt(mean_squared_error(test_y_time.set_index(geography_col).loc[:, outcome_col], pred_df)))
        
        maes.append(mean_absolute_error(test_y_time.set_index(geography_col).loc[:, outcome_col], pred_df))

        xtop_year.append(result)
    annual_avg = np.array(xtop_year).mean(axis=0)
    xtops.append(annual_avg)

In [28]:
xtops

[0.4444444444444446, 0.45290581162324633]

In [29]:
np.mean(xtops)

0.44867512803384546

In [19]:
xtops

[0.4444444444444446, 0.45290581162324633]

In [20]:
np.mean(xtops)

0.44867512803384546

In [21]:
np.mean(maes)

1.1334389263152322

In [22]:
np.mean(rmses)

3.4415631203967694